# Bridge L3: M12.1 → M12.2 Readiness Validation
**Usage Metering → Billing Integration**

This notebook validates bridge readiness between Module 12.1 (Usage Metering) and Module 12.2 (Billing Integration).

## Section 1: M12.1 Accomplishments Recap

### What We Built in M12.1

The previous module delivered five major systems:

1. **ClickHouse Analytics Database**
   - Partitioned tables tracking queries, tokens, and bytes per tenant
   - Real-time data ingestion for usage events

2. **Async Event Tracker with Fallback**
   - Non-blocking event capture (prevents API slowdown)
   - File-based recovery when ClickHouse unavailable

3. **Per-Tenant Cost Attribution**
   - Conversion of usage metrics to USD using OpenAI pricing + markup
   - Costs locked at event time (immutable pricing)

4. **Redis-Based Quota Enforcement**
   - Synchronous quota checks blocking requests at limits
   - Prevents unexpected charges

5. **Real-Time Dashboards**
   - Grafana materialized views
   - Sub-second performance for tenant consumption visibility

### The Problem We Still Have

**Manual billing doesn't scale:**
- 50 customers = 10 hours/month manual work ($1,500 opportunity cost)
- 100 customers = 40+ hours/month (unsustainable)
- Perfect usage data exists, but no automated payment collection

**This bridge validates readiness before automating billing in M12.2.**

## Section 2: Readiness Check #1 - ClickHouse Data Accuracy

**Requirement:** Query monthly aggregations; verify all tenants have usage records.

**Failure Impact:** 3+ hours debugging billing mismatches later.

**Test:** Retrieve last 30 days of tenant usage from ClickHouse.

In [ ]:
import os
from datetime import datetime, timedelta

# Check for ClickHouse connection
CLICKHOUSE_HOST = os.getenv("CLICKHOUSE_HOST", "localhost")
CLICKHOUSE_DB = os.getenv("CLICKHOUSE_DB", "analytics")

try:
    # Stub: Would connect to ClickHouse and query usage
    # from clickhouse_driver import Client
    # client = Client(host=CLICKHOUSE_HOST)
    # query = """
    #   SELECT tenant_id, COUNT(*) as events, SUM(cost_usd) as total_cost
    #   FROM usage_events
    #   WHERE event_date >= today() - 30
    #   GROUP BY tenant_id
    # """
    # result = client.execute(query)
    
    print("⚠️  Skipping (no ClickHouse connection)")
    print("# Expected: List of tenants with event counts and total costs")
    print("# Example output:")
    print("# tenant_123 | 1,450 events | $127.35")
    print("# tenant_456 | 892 events | $76.20")
    
except Exception as e:
    print(f"⚠️  Check skipped: {e}")

## Section 3: Readiness Check #2 - Immutable Cost Storage

**Requirement:** Regenerate prior month's invoice twice; confirm identical totals.

**Failure Impact:** Billing disputes costing $500-2,000 per incident.

**Test:** Generate the same invoice twice and verify deterministic output.

In [ ]:
import hashlib
import json

def generate_invoice_stub(tenant_id, month):
    """Stub invoice generator - would query ClickHouse in production"""
    # Mock data (would come from ClickHouse)
    invoice_data = {
        "tenant_id": tenant_id,
        "month": month,
        "total_queries": 1250,
        "total_tokens": 185000,
        "total_cost_usd": 127.35
    }
    return invoice_data

# Test: Generate invoice twice, verify identical results
tenant_id = "tenant_123"
month = "2025-10"

invoice_run1 = generate_invoice_stub(tenant_id, month)
invoice_run2 = generate_invoice_stub(tenant_id, month)

hash1 = hashlib.md5(json.dumps(invoice_run1, sort_keys=True).encode()).hexdigest()
hash2 = hashlib.md5(json.dumps(invoice_run2, sort_keys=True).encode()).hexdigest()

if hash1 == hash2:
    print("✓ Invoice determinism verified")
    print(f"# Expected: Identical hashes ({hash1[:8]}...)")
else:
    print("✗ FAIL: Invoices differ between runs!")
    
print("# Pass criteria: Same input → same output (±$0.01 rounding)")

## Section 4: Readiness Check #3 - Dashboard Performance

**Requirement:** Load largest tenant's dashboard; measure sub-second response.

**Failure Impact:** Slow dashboards delay customer payments 5-10 days.

**Test:** Simulate dashboard query and measure response time.

In [ ]:
import time

GRAFANA_URL = os.getenv("GRAFANA_URL", None)

if not GRAFANA_URL:
    print("⚠️  Skipping (no Grafana connection)")
    print("# Expected: Dashboard load time < 1.0s")
    print("# Example output:")
    print("# Dashboard loaded in 0.347s ✓")
else:
    # Stub: Would query Grafana API or ClickHouse materialized view
    start_time = time.time()
    
    # Simulate dashboard query
    # response = requests.get(f"{GRAFANA_URL}/api/dashboards/tenant_123")
    time.sleep(0.15)  # Simulate fast query
    
    elapsed = time.time() - start_time
    
    if elapsed < 1.0:
        print(f"✓ Dashboard loaded in {elapsed:.3f}s")
    else:
        print(f"✗ FAIL: Dashboard took {elapsed:.3f}s (> 1.0s target)")
    
    print("# Pass criteria: Response time < 1.0 second")

## Section 5: Readiness Check #4 - Quota Enforcement

**Requirement:** Set test quota at 10 queries; confirm 11th query returns 429 error.

**Failure Impact:** Unexpected overage charges ($200-500 per incident).

**Test:** Simulate quota limit enforcement with Redis-based counter.

In [ ]:
REDIS_HOST = os.getenv("REDIS_HOST", None)

class QuotaEnforcerStub:
    """Simulates Redis-based quota enforcement"""
    def __init__(self, limit):
        self.limit = limit
        self.counter = 0
    
    def check_quota(self, tenant_id):
        self.counter += 1
        if self.counter > self.limit:
            return {"allowed": False, "status": 429, "message": "Quota exceeded"}
        return {"allowed": True, "status": 200}

# Test: Simulate 11 requests with 10-query limit
quota = QuotaEnforcerStub(limit=10)

print("Testing quota enforcement (limit=10):")
for i in range(1, 12):
    result = quota.check_quota("test_tenant")
    if i <= 10:
        assert result["allowed"] == True, f"Request {i} should be allowed"
    else:
        assert result["allowed"] == False, f"Request {i} should be blocked"
        assert result["status"] == 429, "Should return HTTP 429"
        print(f"✓ Request {i}: Blocked with 429 (quota exceeded)")

print("# Expected: Requests 1-10 pass, request 11 returns 429")
print("# Pass criteria: Hard quota block prevents overage charges")

## Section 6: Call-Forward to M12.2 - What's Next

### The Automation Challenge

**Current state:** Perfect usage data exists, but billing requires 10+ hours of manual work monthly at 50 customers.

**The goal of M12.2:** Transform billing into a fully automated process that scales to unlimited customers.

---

### What M12.2 Will Deliver

The next module builds **four automated payment systems:**

#### 1. Stripe Customer & Subscription Setup
- Automatic customer creation from tenant signups
- Plan linking to Stripe price IDs
- Metered billing model mapping (usage → Stripe reporting API)

#### 2. Automated Invoice Generation
- Monthly ClickHouse-to-Stripe export pipeline
- Itemized invoices with line-item breakdowns
- Automatic email distribution to customers

#### 3. Payment Collection & Dunning
- Automatic charge attempts on invoice creation
- Retry logic at days 3, 7, and 14 for failed payments
- Webhook notifications for payment events (success/failure)

#### 4. Subscription Lifecycle Management
- Trial-to-paid conversion workflows
- Mid-month upgrades/downgrades with proration
- Cancellation handling and final invoicing

---

### Business Impact

**Time savings:**
- 50 customers: 10 hours/month → 0 hours (automated)
- 200 customers: 40+ hours/month → 0 hours (automated)
- **Scales to unlimited customers with zero manual intervention**

**Cost reduction:**
- Eliminates $1,500-3,000/month opportunity cost
- Prevents billing disputes ($500-2,000 per incident)
- Accelerates payment collection (5-10 day improvement)

---

### Bridge Validation Complete

**If all four readiness checks passed above, you are ready to proceed to M12.2.**

Next: Build Stripe integration and automate end-to-end billing.